In [2]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from src.dataset import ChestXrayDataSet, CLASS_NAMES
from src.model import DenseNet121
from src.utils import compute_AUCs, compute_score_with_logits, tile
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Autoreload modules so that changes to src automatically reflect
%load_ext autoreload
%autoreload 2

In [3]:
labels = pd.read_csv("data/labels/cleaned.csv")
X, Y = labels.iloc[:, 0], labels.iloc[:, 1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, shuffle=True)

In [4]:
N_CLASSES = len(CLASS_NAMES)
BATCH_SIZE = 64
DATA_DIR = "data/images"

In [5]:
train_dataset = ChestXrayDataSet(data_dir=DATA_DIR, X = X_train, Y = Y_train,
                          transform=transforms.Compose([
                              transforms.Resize(256),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225]),
                              transforms.RandomHorizontalFlip()
                              ]))

# note that workers take up some amount of VRAM   
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

test_dataset = ChestXrayDataSet(data_dir=DATA_DIR, X = X_test, Y = Y_test,
                          transform=transforms.Compose([
                              transforms.Resize(256),
                              transforms.ToTensor(),
                              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                              ]))
    
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

In [6]:
training = False # Flip to false to simply load pre-trained model
CKPT_TRAINED_PATH = "model-trained.pth"

cudnn.benchmark = True # Fixed input size, enables tuning for optimal use

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# initialize and load the model
model = DenseNet121(N_CLASSES).to(device)

if not training:
    if os.path.isfile(CKPT_TRAINED_PATH):
        print("=> loading checkpoint")
        checkpoint = torch.load(CKPT_TRAINED_PATH)
        model.load_state_dict(checkpoint, strict=True)
        print("=> loaded checkpoint")
    else:
        print("=> no checkpoint found")

=> loading checkpoint
=> loaded checkpoint


## Training

In [7]:
# Source: https://github.com/thibaultwillmann/CheXNet-Pytorch/blob/master/CheXnet.ipynb

num_epochs = 10

if training:
    model.train()

    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters())

    for epoch in range(num_epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        correct = 0
        total = 0 
        for i, (images, labels) in enumerate(train_loader, 0): # get the inputs; data is a list of [images, labels]

            # images.shape -> [N, 3, 224, 224]
            # labels.shape -> [N, 14]

            # zero the parameter gradients
            optimizer.zero_grad(set_to_none=True)

            images = images.cuda()

            n_batches, channels, height, width = images.size()
            image_batch = torch.autograd.Variable(images.view(-1, channels, height, width))

            labels = tile(labels, 0, 1).cuda()
                     
            outputs = model(image_batch)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            correct += compute_score_with_logits(outputs, labels).sum()
            total += labels.size(0)

        print('Epoch: %d, loss: %.3f, Accuracy: %.3f' %
            (epoch + 1, running_loss, 100 * correct / total))

    print('Finished Training')

## Evaluation of Trained Model

In [8]:
# initialize the ground truth and output tensor
gt = torch.FloatTensor().to(device)
pred = torch.FloatTensor().to(device)

# switch to evaluate mode
model.eval()

for i, (inp, target) in enumerate(test_loader):
    target = target.cuda()
    gt = torch.cat((gt, target), 0)
    bs, c, h, w = inp.size()
    with torch.no_grad():
        input_var = torch.autograd.Variable(inp.view(-1, c, h, w).cuda())
        output = model(input_var)
        output_mean = output.view(bs, 1, -1).mean(1)
        pred = torch.cat((pred, output_mean.data), 0)

In [9]:
AUROCs = compute_AUCs(gt, pred, N_CLASSES)
AUROC_avg = np.array(AUROCs).mean()
print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
for i in range(N_CLASSES):
  print('The AUROC of {} is {}'.format(CLASS_NAMES[i], AUROCs[i]))

The average AUROC is 0.832
The AUROC of Atelectasis is 0.8439077879845739
The AUROC of Cardiomegaly is 0.9556624963808421
The AUROC of Effusion is 0.8795635817106547
The AUROC of Infiltration is 0.78179352343567
The AUROC of Mass is 0.8491913647572664
The AUROC of Nodule is 0.8024044557440636
The AUROC of Pneumonia is 0.6726941681613131
The AUROC of Pneumothorax is 0.8660213682820934
The AUROC of Consolidation is 0.7400722681767539
The AUROC of Edema is 0.8802701754573411
The AUROC of Emphysema is 0.8807207977546327
The AUROC of Fibrosis is 0.8253416567675599
The AUROC of Pleural_Thickening is 0.7445453324703031
The AUROC of Hernia is 0.9267122239328704


In [16]:
if training:
    torch.save(model.state_dict(), "model-trained.pth")